# Spark User Defined Functions

## Overview

Spark provides a powerful computational engine and we have seen some core aspects of it thus far. In this
section, we will look into how to create user defined functions and integrate these in Spark.